In [ ]:
class LSTMAutoencoder(tf.keras.Model):
    """
    An LSTM autoencoder model for time series
    """
    def __init__(
        self,
        n_latent,
        time_window,
        n_features=1,
        network_shape=[],
        latent_regularizer=None,
        rnn_opts=dict(),
        activation_func=tf.keras.layers.ELU(alpha=1.0),
        random_state=None,
        **kwargs
    ):
        super(LSTMAutoencoder, self).__init__()
        self.n_latent = n_latent
        self.time_window = time_window
        self.n_features = n_features
        
        # Initialize state
        tf.random.set_seed(random_state)
        
        # Encoder
        self.encoder = tf.keras.Sequential()
        self.encoder.add(tf.keras.layers.InputLayer(input_shape=(time_window, n_features)))
        self.encoder.add(tf.keras.layers.GaussianNoise(0.5)) # smooths the output
        
        for i, hidden_size in enumerate(network_shape):
            self.encoder.add(
                tf.keras.layers.LSTM(
                    hidden_size,
                    #input_shape=(time_window, n_features),
                    return_sequences=True,
                    name="lstm_encoder_"+str(i),
                    **rnn_opts
                )
            )
            self.encoder.add(tf.keras.layers.BatchNormalization())
            self.encoder.add(tf.keras.layers.Activation(activation_func))
        self.encoder.add(
            tf.keras.layers.LSTM(
                n_latent,
                #input_shape=(time_window, n_features),
                return_sequences=False,
                name="lstm_encoder_final",
                **rnn_opts
            )
        )
        self.encoder.add(tf.keras.layers.BatchNormalization(activity_regularizer=latent_regularizer))
            
        ## Decoder
        self.decoder = tf.keras.Sequential()
        self.decoder.add(tf.keras.layers.GaussianNoise(0.5, input_shape=(n_latent,)))
        self.decoder.add(tf.keras.layers.RepeatVector(time_window))
        self.decoder.add(
            tf.keras.layers.LSTM(
                n_latent,
                #input_shape=(time_window, n_features),
                return_sequences=True,
                name="lstm_decoder_initial",
                **rnn_opts
            )
        )
        
        
        for i, hidden_size in enumerate(network_shape[::-1]):
            self.decoder.add(
                tf.keras.layers.LSTM(
                    hidden_size, 
                    return_sequences=True, 
                    go_backwards=True, 
                    name="lstm_decoder_"+str(i),
                    **rnn_opts
                )
            )
        self.decoder.add(
            tf.keras.layers.LSTM(
                n_features, 
                return_sequences=True, 
                go_backwards=True, 
                name="lstm_decoder_final"),
                **rnn_opts
        )
        self.decoder.add(tf.keras.layers.BatchNormalization())
        #self.decoder.add(tf.keras.layers.Activation(activation_func))
        
    def call(self, inputs, training=False):
        outputs = self.decoder(self.encoder(inputs))
        return outputs
this is called in the below code

class TimeSeriesEmbedding:
    """Base class for time series embedding
    
    Properties
    ----------
    
    train_history : dict
        The training history of the model
    
    model : "lstm" | "mlp" | "tica" | "etd" | "delay"
        The type of model to use for the embedding.
        
    n_latent : int
        The embedding dimension
        
    n_features : int
        The number of channels in the time series
    
    **kwargs : dict
        Keyword arguments passed to the model
        
    """
    def __init__(
        self, 
        n_latent,
        time_window=10, 
        n_features=1, 
        random_state=None,
        **kwargs
    ):
        self.n_latent = n_latent
        self.time_window = time_window
        self.n_features = n_features
        self.random_state = random_state
        
    
    def fit(self, X, y=None):
        raise AttributeError("Derived class does not contain method.")
           
    def transform(self, X, y=None):
        raise AttributeError("Derived class does not contain method.")

    def fit_transform(self, X, y=None, **kwargs):
        """Fit the model with a time series X, and then embed X.

        Parameters
        ----------
        X : array-like, shape (n_timepoints, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.

        y : None
            Ignored variable.
            
        kwargs : keyword arguments passed to the model's fit() method

        Returns
        -------
        X_new : array-like, shape (n_timepoints, n_components)
            Transformed values.
        """
        self.fit(X, **kwargs)
        return self.transform(X)
class NeuralNetworkEmbedding(TimeSeriesEmbedding):
    """Base class autoencoder model for time series embedding
    
    Properties
    ----------
    
    n_latent : int
        The embedding dimension
        
    n_features : int
        The number of channels in the time series
    
    **kwargs : dict
        Keyword arguments passed to the model
        
    """
    
    def __init__(
        self,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        # # Default latent regularizer is FNN
        # if np.isscalar(latent_regularizer):
        #     latent_regularizer = FNN(latent_regularizer)
    
    def fit(
        self, 
        X,
        y=None,
        subsample=None,
        tau=0,
        learning_rate=1e-3, 
        batch_size=100, 
        train_steps=200,
        loss='mse',
        verbose=0,
        optimizer="adam",
        early_stopping=False
    ):
        """Fit the model with a time series X

        Parameters
        ----------
        X : array-like, shape (n_timepoints, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.

        y : None
            Ignored variable.

        subsample : int or None
            If set to an integer, a random number of timepoints is selected
            equal to that integer
            
        tau : int
            The prediction time, or the number of timesteps to skip between 
            the input and output time series


        Returns
        -------
        X_new : array-like, shape (n_timepoints, n_components)
            Transformed values.
        """
        # Make hankel matrix from dataset
        Xs = standardize_ts(X)
        
        # X_train = hankel_matrix(Xs, self.time_window)
        # Split the hankel matrix for a prediction task
        X0 = hankel_matrix(Xs, self.time_window + tau)
        X_train = X0[:, :self.time_window ]
        Y_train = X0[:, -self.time_window:]
        
        
        if subsample:
            self.train_indices, _ = resample_dataset(
                X_train, subsample, random_state=self.random_state
            )
            X_train = X_train[self.train_indices]
            Y_train = Y_train[self.train_indices]


        optimizers = {
            "adam": tf.keras.optimizers.Adam(learning_rate=learning_rate),
            "nadam": tf.keras.optimizers.Nadam(learning_rate=learning_rate)
            # "radam": tfa.optimizers.RectifiedAdam(learning_rate=learning_rate),
        }

        tf.random.set_seed(self.random_state)
        np.random.seed(self.random_state)
        self.model.compile(
            optimizer=optimizers[optimizer], 
            loss=loss,
            #experimental_run_tf_function=False
        )    
        
        if early_stopping:
            callbacks = [tf.keras.callbacks.EarlyStopping(monitor='loss', mode='min', patience=3)]
        else:
            callbacks = [None]
        
        self.train_history = self.model.fit(
            x=tf.convert_to_tensor(X_train),                         
            y=tf.convert_to_tensor(Y_train),
            epochs=train_steps,
            batch_size=batch_size,
            verbose=verbose
        )
             
    def transform(self, X, y=None):
        X_test = hankel_matrix(standardize_ts(X), self.time_window)
        X_new = self.model.encoder.predict(X_test)
        return X_new 

class LSTMEmbedding(NeuralNetworkEmbedding): 
    def __init__(
        self,
        *args,
        **kwargs
    ):
        super().__init__(*args, **kwargs)
        kwargs.pop("time_window")
        if use_legacy:
            self.model = LSTMAutoencoderLegacy(
                self.n_latent,
                self.time_window,
                **kwargs
            ) 
        else:
            self.model = LSTMAutoencoder(
                self.n_latent,
                self.time_window,
                **kwargs
            )
